In [1]:
!pip3 install torch
!pip3 install torchvision
!pip3 install numpy

import torch
import random
import numpy as np
import torch as torch
from torch.utils.data import dataloader, dataset
import torchvision
#
# random.seed(0)
# np.random.seed(0)
# torch.manual_seed(0)
# torch.cuda.manual_seed(0)
from torch.utils.data.sampler import SubsetRandomSampler
from torchvision.models.detection import transform
from torchvision import transforms
torch.backends.cudnn.deterministic = True
# import pycuda.autoinit
# import pycuda.driver as drv

^C
ERROR: Operation cancelled by user
  Using cached torchvision-0.7.0-cp38-cp38-manylinux1_x86_64.whl (5.9 MB)
  Using cached torch-1.6.0-cp38-cp38-manylinux1_x86_64.whl (748.8 MB)
     |████████████████████████████████| 14.5 MB 11.0 MB/s eta 0:00:01
     |████████████████████████████████| 2.2 MB 10.9 MB/s eta 0:00:01
  Using cached future-0.18.2.tar.gz (829 kB)
  Created wheel for future: filename=future-0.18.2-py3-none-any.whl size=491059 sha256=4f22cce3c143bc2114237b3e5ca45a8a197d60360a1b632d8fe18fc47a5f91ce
  Stored in directory: /home/alexsun/.cache/pip/wheels/8e/70/28/3d6ccd6e315f65f245da085482a2e1c7d14b90b30f239e2cf4
Successfully built future


In [2]:
# CUDA for PyTorch
use_cuda = torch.cuda.is_available()
device = torch.device("cuda:0" if use_cuda else "cpu")
torch.backends.cudnn.benchmark = True
torch.backends.cudnn.enabled

True

In [3]:
import csv
import os
from re import search

def make_img_folder(dataPath, csvPath, imgFolder):
    with open(csvPath) as f:
        reader = csv.reader(f)
        for row in reader:
            if row[0].isdigit():
                print(f"cp {dataPath}/{'.'.join([row[0],'png'])} {imgFolder}/{row[1]}/")
                os.system(f"mv {dataPath}/{'.'.join([row[0],'png'])} {imgFolder}/{row[1]}/")
Data_path = "/home/alexsun/diskb/streamlabs/task/images"
csv_path = "/home/alexsun/diskb/streamlabs/task/images_labelling.csv"
imgFolder = "/home/alexsun/diskb/streamlabs/task/data"
# make_img_folder(Data_path, csv_path, imgFolder)

In [4]:
from __future__ import print_function, division

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy

plt.ion()   # interactive mode

ModuleNotFoundError: No module named 'matplotlib'

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

data_dir = '/content/gdrive/My Drive/data'
image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x),
                                          data_transforms[x])
                  for x in ['train', 'val']}
dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=4,
                                             shuffle=True, num_workers=4)
              for x in ['train', 'val']}
dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}
class_names = image_datasets['train'].classes

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=25):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
            if phase == 'train':
                scheduler.step()

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model

In [ ]:
model_ft = models.resnet18(pretrained=True)
num_ftrs = model_ft.fc.in_features
# Here the size of each output sample is set to 2.
# Alternatively, it can be generalized to nn.Linear(num_ftrs, len(class_names)).
model_ft.fc = nn.Linear(num_ftrs, 2)

model_ft = model_ft.to(device)

criterion = nn.CrossEntropyLoss()

# Observe that all parameters are being optimized
optimizer_ft = optim.SGD(model_ft.parameters(), lr=0.001, momentum=0.9)

# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)

In [ ]:
model_ft = train_model(model_ft, criterion, optimizer_ft, exp_lr_scheduler,
                       num_epochs=25)

In [ ]:
def visualize_model(model, num_images=6):
    was_training = model.training
    model.eval()
    images_so_far = 0
    fig = plt.figure()

    with torch.no_grad():
        for i, (inputs, labels) in enumerate(dataloaders['val']):
            inputs = inputs.to(device)
            labels = labels.to(device)

            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)

            for j in range(inputs.size()[0]):
                images_so_far += 1
                ax = plt.subplot(num_images//2, 2, images_so_far)
                ax.axis('off')
                ax.set_title('predicted: {}'.format(class_names[preds[j]]))
                imshow(inputs.cpu().data[j])

                if images_so_far == num_images:
                    model.train(mode=was_training)
                    return
        model.train(mode=was_training)


visualize_model(model_conv)

plt.ioff()
plt.show()

In [ ]:
dataset = torchvision.datasets.ImageFolder(imgFolder,transform = transforms.Compose([transforms.Resize((256,256)),transforms.ToTensor()]))
batch_size = 32
validation_split = .3
shuffle_dataset = True
random_seed= 8

dataset_size = len(dataset)
indices = list(range(dataset_size))
split = int(np.floor(validation_split * dataset_size))
if shuffle_dataset :
    np.random.seed(random_seed)
    np.random.shuffle(indices)
train_indices, val_indices = indices[split:], indices[:split]

train_sampler = SubsetRandomSampler(train_indices)
valid_sampler = SubsetRandomSampler(val_indices)

train_loader = torch.utils.data.DataLoader(dataset, batch_size=batch_size,
                                           sampler=train_sampler)
validation_loader = torch.utils.data.DataLoader(dataset,
                                                sampler=valid_sampler)
dataset

In [ ]:
from torchvision import datasets, transforms as T
import torch

model = torch.hub.load('pytorch/vision:v0.6.0', 'resnet18', pretrained=True)
model.eval()

# first_conv_layer = [torch.nn.Conv2d(1, 3, kernel_size=3, stride=1, padding=1, dilation=1, groups=1, bias=True)]
# first_conv_layer.extend(list(model.children()))
# model.features= torch.nn.Sequential(*first_conv_layer)
# output = model(x)
# modules = list(model.children())[:-1]      # delete the last fc layer.
# model = torch.nn.Sequential(*first_conv_layer)

loss = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1.0e-3)

# model_ft, hist = train_model(model_ft, dataloaders_dict, criterion, optimizer_ft, num_epochs=num_epochs, is_inception=(model_name=="inception"))

for epoch in range(10000):
    for i, btch in enumerate(train_loader):
        x_train, labels_train = btch
        # resize = torchvision.transforms.Resize(x_train[0].shape)
        # x_train = resize(x_train)
        x_train = torch.Tensor(x_train).float()
        optimizer.zero_grad()

        preds = model.forward(x_train)

        loss_value = loss(preds, labels_train)
        loss_value.backward()

        optimizer.step()

    # acc = []
    x_test, target = validation_loader
    test_preds = model.forward(x_test)
    # test_loss_history.append(loss(test_preds, target))
    acc=(test_preds.argmax(dim=1) == target).float()
    # train_preds = mnist_net.forward(X_train)
    # train_loss_history.append(loss(train_preds, y_train))

    # print(acc)
    accuracy =acc.mean()
    # accuracy = (test_preds.argmax(dim=1) == y_test).float().mean()
    # accuracy = (train_preds.argmax(dim=1) == y_train).float().mean()
    # test_accuracy_history.append(accuracy)
    # train_accuracy_history.append(accuracy)
    print(accuracy)
